<h2 id="Contents">Contents<a href="#Contents"></a></h2>
        <ol>
        <ol><li><a class="" href="#Exceptions">Exceptions</a></li>
<li><a class="" href="#Raising-exceptions">Raising exceptions</a></li>
<ol><li><a class="" href="#The-effects-of-an-exception">The effects of an exception</a></li>
</ol><li><a class="" href="#Handling-exceptions">Handling exceptions</a></li>
<li><a class="" href="#The-exception-hierarchy">The exception hierarchy</a></li>
<li><a class="" href="#Defining-our-own-exceptions">Defining our own exceptions</a></li>
<li><a class="" href="#Case-study">Case study</a></li>
</ol>

## Exceptions

>When an exception occurs, everything that was supposed to happen doesn't
happen, unless it was supposed to happen when an exception occurred.

* How to cause an exception to occur
* How to recover when an exception has occurred
* How to handle different exception types in different ways
* Cleaning up when an exception has occurred
* Creating new types of exception
* Using the exception syntax for flow control

## Raising exceptions

In Python, the words error and exception are used almost interchangeably.
Errors are sometimes considered more dire than exceptions, but they are dealt with
in exactly the same way. Indeed, all the error classes have
`Exception` (which extends `BaseException`) as their superclass.

In [1]:
class EvenOnly(list):
    def append(self, integer):
        if not isinstance(integer, int):
            raise TypeError("Only integers can be added")
        if integer % 2:
            raise ValueError("Only even numbers can be added")
        super().append(integer)

list_of_evens = EvenOnly()
list_of_evens.append(3)

ValueError: Only even numbers can be added

To raise exception, we can use the `raise` keyword. The `raise` keyword is simply
followed by the object being raised as an exception. In the preceding example, two
objects are newly constructed from the built-in classes `TypeError` and `ValueError`.

### The effects of an exception
When an exception is raised, it appears to stop program execution immediately.
Any lines that were supposed to run after the exception is raised are not executed,
and unless the exception is dealt with, the program will exit with an error message.

In [2]:
def no_return():
    print("I am about to raise an exception")
    raise Exception("This is always raised")
    print("This line will never execute")
    return "I won't be returned"

no_return()

I am about to raise an exception


Exception: This is always raised

## Handling exceptions

We handle exceptions
by wrapping any code that might throw one (whether it is exception code itself, or
a call to any function or method that may have an exception raised inside it) inside
a `try...except` clause. 

In [3]:
try:
    no_return()
except:
    print("I caught an exception")
    print("executed after the exception")

I am about to raise an exception
I caught an exception
executed after the exception


We can use `except ExceptionName` to handle exceptions of a specific type. We can even use `except (ExceptionName1, ExceptionName2)` to handle multiple types of exceptions.

In [4]:
def funny_division(divider):
    try:
        return 100 / divider
    except ZeroDivisionError:
        return "Zero is not a good idea!"
print(funny_division(0))
print(funny_division(50.0))
print(funny_division("hello"))

Zero is not a good idea!
2.0


TypeError: unsupported operand type(s) for /: 'int' and 'str'

Finally, we can use `raise` again in an except block to raise the exception.

In [6]:
def funny_division3(anumber):
    try:
        if anumber == 13:
            raise ValueError("13 is an unlucky number")
        return 100 / anumber
    except ZeroDivisionError:
        return "Enter a number other than zero"
    except TypeError:
        return "Enter a numerical value"
    except ValueError:
        print("No, No, not 13!")
        raise

funny_division3(13)

No, No, not 13!


ValueError: 13 is an unlucky number

Sometimes, when we catch an exception, we need a reference to the `Exception` object
itself. This most often happens when we define our own exceptions with custom
arguments, but can also be relevant with standard exceptions. Most exception classes
accept a set of arguments in their constructor, and we might want to access those
attributes in the exception handler. If we define our own exception class, we can even
call custom methods on it when we catch it. The syntax for capturing an exception as
a variable uses the `as` keyword:

In [7]:
try:
    raise ValueError("This is an argument")
except ValueError as e:
    print("The exception arguments were", e.args)

The exception arguments were ('This is an argument',)


`finally` and `else` clauses are also useful for dealing with exceptions. The `finally` clause is executed no matter what
happens. This is extremely useful when we need to perform certain tasks after
our code has finished running (even if an exception has occurred). <br>
The `finally` clause is also very important when we execute a `return` statement
from inside a `try` clause. The `finally` handle will still be executed before the
value is returned.

In [10]:
import random
some_exceptions = [ValueError, TypeError, IndexError, None]
try:
    choice = random.choice(some_exceptions)
    print("raising {}".format(choice))
    if choice:
        raise choice("An error")
except ValueError:
    print("Caught a ValueError")
except TypeError:
    print("Caught a TypeError")
except Exception as e:
    print("Caught some other error: %s" %
( e.__class__.__name__))
else:
    print("This code called if there is no exception")
finally:
    print("This cleanup code is always called")

raising <class 'IndexError'>
Caught some other error: IndexError
This cleanup code is always called


>Any of the `except`, `else`, and `finally` clauses can be omitted after a `try` block
(although else by itself is invalid). If you include more than one, the `except` clauses
must come first, then the `else` clause, with the `finally` clause at the end. The order
of the `except` clauses normally goes from most specific to most generic.

## The exception hierarchy

Most exceptions are subclasses of the `Exception` class. But
this is not true of all exceptions. `Exception` itself actually inherits from a class called
`BaseException`. In fact, all exceptions must extend the `BaseException` class or one
of its subclasses.
There are two key exceptions, `SystemExit` and `KeyboardInterrupt`, that derive
directly from `BaseException` instead of `Exception`.

<img src="img/04_01.png">

>When we use the `except:` clause without specifying any type of exception, it will
catch all subclasses of `BaseException`; which is to say, it will catch all exceptions,
including the two special ones. Since we almost always want these to get special
treatment, it is unwise to use the `except:` statement without arguments. If you want
to catch all exceptions other than SystemExit and KeyboardInterrupt, explicitly
catch `Exception`.

## Defining our own exceptions

To define our own exceptions, all we have to do is inherit from the `Exception` class. We don't even have to add any
content to the class! We can, of course, extend `BaseException` directly, but then it
will not be caught by generic except `Exception` clauses.

In [11]:
class InvalidWithdrawal(Exception):
    pass
raise InvalidWithdrawal("You don't have $50 in your account")

InvalidWithdrawal: You don't have $50 in your account

Of course, if we do want to customize the initializer, we are free to do so. Here's
an exception whose initializer accepts the current balance and the amount the user
wanted to withdraw. In addition, it adds a method to calculate how overdrawn the
request was:

In [12]:
class InvalidWithdrawal(Exception):
    def __init__(self, balance, amount):
        super().__init__("account doesn't have ${}".format(
        amount))
        self.amount = amount
        self.balance = balance
    def overage(self):
        return self.amount - self.balance
raise InvalidWithdrawal(25, 50)

InvalidWithdrawal: account doesn't have $50

To use its full potential:

In [13]:
try:
    raise InvalidWithdrawal(25, 50)
except InvalidWithdrawal as e:
    print("I'm sorry, but your withdrawal is "
    "more than your balance by "
    "${}".format(e.overage()))

I'm sorry, but your withdrawal is more than your balance by $25


In [14]:
def divide_with_exception(number, divisor):
    try:
        print("{} / {} = {}".format(
        number, divisor, number / divisor * 1.0))
    except ZeroDivisionError:
        print("You can't divide by zero")


def divide_with_if(number, divisor):
    if divisor == 0:
        print("You can't divide by zero")
    else:
        print("{} / {} = {}".format(
        number, divisor, number / divisor * 1.0))

These two functions behave identically. If divisor is zero, an error message is
printed; otherwise, a message printing the result of division is displayed. We
could avoid a `ZeroDivisionError` ever being thrown by testing for it with an `if`
statement. Similarly, we can avoid an `IndexError` by explicitly checking whether
or not the parameter is within the confines of the list, and a `KeyError` by checking
if the key is in a dictionary.<br>
But we shouldn't do this. For one thing, we might write an if statement that checks
whether or not the index is lower than the parameters of the list, but forget to check
negative values.

## Case study

Here, we'll be designing a simple central authentication and authorization system.
The entire system will be placed in one module, and other code will be able to query
that module object for authentication and authorization purposes. We should admit,
from the start, that we aren't security experts, and that the system we are designing
may be full of security holes. Our purpose is to study exceptions, not to secure a
system. It will be sufficient, however, for a basic login and permission system that
other code can interact with

In [15]:
import hashlib

In [16]:
hash_string = ("Hello World!")
hash_string = hash_string.encode("utf8")
hashlib.sha256(hash_string).hexdigest()

'7f83b1657ff1fc53b92dc18148a1d65dfc2d4b1fa3d677284addd200126d9069'

In [18]:
hash_string = ("Hello world!")
hash_string = hash_string.encode("utf8")
hashlib.sha256(hash_string).hexdigest()

'c0535e4be2b79ffd93291305436bf889314e4a3faec05ecffcbb7df31ad9e51a'

In [25]:
cd "Case Study Ch.4"

[WinError 2] The system cannot find the file specified: 'Case Study Ch.4'
c:\Users\harik\Desktop\Python\Notes\OOP\Case Study Ch. 4


In [26]:
pwd

'c:\\Users\\harik\\Desktop\\Python\\Notes\\OOP\\Case Study Ch. 4'

In [27]:
import auth

In [28]:
auth.authenticator.add_user("joe", "joepassword")

In [29]:
auth.authorizor.add_permission("paint")
auth.authorizor.check_permission("paint", "joe")

NotLoggedInError: ('joe', None)

In [30]:
auth.authenticator.is_logged_in("joe")

False

In [32]:
auth.authenticator.login("joe", "joepassword")

True

In [33]:
auth.authorizor.check_permission("paint", "joe")

NotPermittedError: ('joe', None)

In [34]:
auth.authorizor.check_permission("mix", "joe")

PermissionError: Permission does not exist

In [36]:
# auth.authenticator.add_user("joe", "joepassword")
auth.authorizor.add_permission("test program")
auth.authorizor.add_permission("change program")
auth.authorizor.permit_user("test program", "joe")